<a href="https://www.kaggle.com/code/nasiyeyldrm/ab-testi-ile-bidding-y-ntemlerini-kar-la-t-rma?scriptVersionId=111984717" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması


![](https://www.invespcro.com/blog/images/blog-images/ab-test-1-1.jpg)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ab-testing/ab_testing.xlsx


In [2]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
# İş Problemi
# Facebook kısa süre önce mevcut "maximum bidding" adı verilen
# teklif verme türüne alternatif olarak yeni bir teklif türü olan
# "average bidding"’i tanıttı.
# Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test
# etmeye karar verdi ve average bidding'in maximum bidding'den
# daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B
# testi yapmak istiyor.
# A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden
# bu A/B testinin sonuçlarını analiz etmenizi bekliyor.
# Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu
# nedenle, istatistiksel testler için Purchase metriğine
# odaklanılmalıdır.

# Veri Seti Hikayesi
# Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra
# buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri
# ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average
# Bidding uygulanmıştır.

## Görev 1: Veriyi Hazırlama ve Analiz Etme

In [4]:
# Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

dfc = pd.read_excel("/kaggle/input/ab-testing/ab_testing.xlsx", sheet_name= "Control Group")

dft = pd.read_excel("/kaggle/input/ab-testing/ab_testing.xlsx", sheet_name= "Test Group")

In [5]:
# Adım 2: Kontrol ve test grubu verilerini analiz ediniz.
dft.head()
dfc.head()

dft.describe().T
dfc.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,101711.44907,20302.15786,45475.94296,85726.69035,99790.70108,115212.81654,147539.33633
Click,40.00000,5100.65737,1329.98550,2189.75316,4124.30413,5001.22060,5923.80360,7959.12507
Purchase,40.00000,550.89406,134.10820,267.02894,470.09553,531.20631,637.95709,801.79502
Earning,40.00000,1908.56830,302.91778,1253.98952,1685.84720,1975.16052,2119.80278,2497.29522


In [6]:
#Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

dfc["Bidding"] = "Maximum_Bidding"

dfc.groupby("Bidding").agg({"Purchase": "mean"})

dft["Bidding"] = "Average_Bidding"

dft.groupby("Bidding").agg({"Purchase": "mean"})

df = pd.concat([dfc, dft])

df.head()

,Impression,Click,Purchase,Earning,Bidding
0,82529.45927,6090.07732,665.21125,2311.27714,Maximum_Bidding
1,98050.45193,3382.86179,315.08489,1742.80686,Maximum_Bidding
2,82696.02355,4167.96575,458.08374,1797.82745,Maximum_Bidding
3,109914.40040,4910.88224,487.09077,1696.22918,Maximum_Bidding
4,108457.76263,5987.65581,441.03405,1543.72018,Maximum_Bidding


## Görev 2: A/B Testinin Hipotezinin Tanımlanması

In [7]:
# Adım 1: Hipotezi tanımlayınız.
# H0 : M1 = M2
# H1 : M1!= M2 

# H0 : M1 = M2  # Average bidding ve Maksimum Bidding purchase ortalamaları Arasında İst. Olarak An. Fark yoktur
# H1 : M1!= M2  # ..... vardır

In [8]:
# Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.

df.groupby("Bidding").agg({"Purchase": "mean"})

,Purchase
Bidding,
Average_Bidding,582.10610
Maximum_Bidding,550.89406


## Görev 3: Hipotez Testinin Gerçekleştirilmesi

In [9]:
# Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.

# Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni
# üzerinden ayrı ayrı test ediniz.

# Normallik Varsayımı :
# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1: Normal dağılım varsayımı sağlanmamaktadır.
# p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
# Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? Elde edilen p-value değerlerini yorumlayınız.

test_stat, pvalue = shapiro(df.loc[df["Bidding"] == "Maximum_Bidding", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [10]:
## p-value = 0.5891 ----> H0'ı reddedemedik, normal dağılıyor

In [11]:
test_stat, pvalue = shapiro(df.loc[df["Bidding"] == "Average_Bidding", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


In [12]:
## p-value = 0.1541 ----> H0'ı reddedemedik, normal dağılıyor

In [13]:
# Varyans Homojenliği :
# H0: Varyanslar homojendir.
# H1: Varyanslar homojen Değildir.
# p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
# Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediniz.
# Test sonucuna göre normallik varsayımı sağlanıyor mu? Elde edilen p-value değerlerini yorumlayınız.

test_stat, pvalue = levene(df.loc[df["Bidding"] == "Maximum_Bidding", "Purchase"],
                           df.loc[df["Bidding"] == "Average_Bidding", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [14]:
## p-value = 0.1083 ----> H0'ı reddedemedik, varyanslar homojen.

In [15]:
# Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

# Her iki varsayım da sağlanıyor. Yani bağımsız iki örneklem t testi yapacağız. (parametrik test)

In [16]:
# Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki olarak anlamlı bir fark olup olmadığını yorumlayınız.

test_stat, pvalue = ttest_ind(df.loc[df["Bidding"] == "Maximum_Bidding", "Purchase"],
                           df.loc[df["Bidding"] == "Average_Bidding", "Purchase"],
                            equal_var=True)
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


In [17]:
## p-value değeri 0.3493 geldi. Yani H0 hipotezini reddedemiyoruz. Yani her iki yöntem arasında anlamlı bir fark var diyemeyiz. H0: M1 = M2 reddedilemez.

## Görev 4: Sonuçların Analizi

In [18]:
# Adım 1: Hangi testi kullandınız, sebeplerini belirtiniz.

#Varsayımlar sağlandığı için bağımsız iki örneklem t testi yaptık.

In [19]:
# Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.

#Elde ettiğimiz sonuca göre average bidding ile maximum bidding arasında purchase ortalamaları açısından anlamlı bir fark olmadığını gördük.
# Müşteriden veri sayısını arttırmasını isteyebiliriz. 3 ay daha veri toplamasını ve tekrar hipotez testi yapılmasını isteyebiliriz.
# Mevcut sonuçlara göre yeni teklif türü olan average biddingi satın almamasını tavsiye ederiz.